In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import OrderedDict
import time
import datetime

In [3]:
!pip install azure.eventhub

    100% |████████████████████████████████| 61kB 3.4MB/s ta 0:00:01
    100% |████████████████████████████████| 2.8MB 474kB/s ta 0:00:011


In [4]:
from azure.eventhub import EventHubClient, Sender, EventData

In [5]:
# rise function
def feature_up(min, max, N, cte_t, noise_amplitude):
    x = np.arange(1.0, N+1)
    y = min + (max - min)*np.exp(-(N - x)/cte_t)
    y = y + noise_amplitude*np.random.normal(size=N)
    return y

# down function
def feature_down(max, min, N, cte_t, noise_amplitude):
    x = np.arange(1.0, N+1)
    y = max - (max - min)*np.exp(-(N - x)/cte_t)
    y = y + noise_amplitude*np.random.normal(size=N)
    return y

# dual function
def feature_dual(min_ciclo1, max_ciclo1, threshold, N, cte_t, noise_amplitude):
    # 0 < threshold < 1
    x = np.arange(1.0, N+1)
    val_ciclo1 = np.array([np.random.uniform(min_ciclo1, max_ciclo1)])
    limite = min_ciclo1 + (max_ciclo1 - min_ciclo1)*threshold
    rango_cicloN = 0.025*np.mean([min_ciclo1, max_ciclo1])
    if val_ciclo1 <= limite:
        # down
        y = val_ciclo1 - rango_cicloN*threshold*np.random.uniform(0, 1)*np.exp(-(N - x)/cte_t)
    else:
        # up
        y = val_ciclo1 + rango_cicloN*(1 - threshold)*np.random.uniform(0, 1)*np.exp(-(N - x)/cte_t)
    
    y = y + noise_amplitude*np.random.normal(size=N)
    return y

In [6]:
# 7 FEATURES: 4 rising (una discreta), 2 decreasing, 1 dual
MIN_MAX = np.array([[641.0, 644.0], 
                   [555.0, 551.0],
                   [46.8 ,  48.5],
                   [388.0, 400.0],
                   [1380.0, 1440.0],
                   [39.5, 38.1],
                   [8110.0, 8150.0]])

# INITIALIZE THE DATASET
dataset = pd.DataFrame()

N_features = MIN_MAX.shape[0]
window_TTF = 10
N_ENGINES = 1#10

In [7]:
for engine_ID in range(1, N_ENGINES + 1):
    
    #N = np.random.randint(100, 300) # failures (dist. uniform), different for each ENGINE
    N = round(np.random.triangular(120, 180, 300)) # failures (dist. triang.), different for each ENGINE
    matrix_features = np.zeros((N, N_features + 4))
    matrix_features[:, 0] = engine_ID
    matrix_features[:, 1] = np.arange(1, N+1)
    matrix_features[:, N_features + 2] = N - matrix_features[:, 1] # RUL
    iterable = (1 if rul < window_TTF else 0 for rul in matrix_features[:, N_features + 2])
    matrix_features[:, N_features + 3] = np.fromiter(iterable, 'int32') # TTF

    # FEATURE 1: RISING
    i = 0
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    peak_min = min_ciclo1 + 0*(max_ciclo1 - min_ciclo1)
    min_i = np.random.triangular(min_ciclo1, peak_min, 0.5*(min_ciclo1 + max_ciclo1))
    peak_max = min_ciclo1 + 1*(max_ciclo1 - min_ciclo1)
    max_i = np.random.triangular(0.5*(min_ciclo1 + max_ciclo1), peak_max, max_ciclo1)
    min_max = np.array([min_i, max_i])
    noise_amplitude = 0.15*np.absolute(np.amax(min_max) - np.amin(min_max))
    matrix_features[:, i+2] = feature_up(np.amin(min_max), np.amax(min_max), N, N*0.2, noise_amplitude)

    # FEATURE 2: DECREASING
    i = 1
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    peak_min = min_ciclo1 + 0*(max_ciclo1 - min_ciclo1)
    min_i = np.random.triangular(min_ciclo1, peak_min, 0.5*(min_ciclo1 + max_ciclo1))
    peak_max = min_ciclo1 + 1*(max_ciclo1 - min_ciclo1)
    max_i = np.random.triangular(0.5*(min_ciclo1 + max_ciclo1), peak_max, max_ciclo1)
    min_max = np.array([min_i, max_i])
    noise_amplitude = 0.15*np.absolute(np.amax(min_max) - np.amin(min_max))
    matrix_features[:, i+2] = feature_down(np.amax(min_max), np.amin(min_max), N, N*0.2, noise_amplitude)

    # FEATURE 3: RISING
    i = 2
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    peak_min = min_ciclo1 + 0*(max_ciclo1 - min_ciclo1)
    min_i = np.random.triangular(min_ciclo1, peak_min, 0.5*(min_ciclo1 + max_ciclo1))
    peak_max = min_ciclo1 + 1*(max_ciclo1 - min_ciclo1)
    max_i = np.random.triangular(0.5*(min_ciclo1 + max_ciclo1), peak_max, max_ciclo1)
    min_max = np.array([min_i, max_i])
    noise_amplitude = 0.15*np.absolute(np.amax(min_max) - np.amin(min_max))
    matrix_features[:, i+2] = feature_up(np.amin(min_max), np.amax(min_max), N, N*0.2, noise_amplitude)

    # FEATURE 4: RISING
    i = 3
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    peak_min = min_ciclo1 + 0*(max_ciclo1 - min_ciclo1)
    min_i = np.random.triangular(min_ciclo1, peak_min, 0.5*(min_ciclo1 + max_ciclo1))
    peak_max = min_ciclo1 + 1*(max_ciclo1 - min_ciclo1)
    max_i = np.random.triangular(0.5*(min_ciclo1 + max_ciclo1), peak_max, max_ciclo1)
    min_max = np.array([min_i, max_i])
    noise_amplitude = 0.15*np.absolute(np.amax(min_max) - np.amin(min_max))
    matrix_features[:, i+2] = np.rint(feature_up(np.amin(min_max), np.amax(min_max), N, N*0.2, noise_amplitude))

    # FEATURE 5: RISING
    i = 4
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    peak_min = min_ciclo1 + 0*(max_ciclo1 - min_ciclo1)
    min_i = np.random.triangular(min_ciclo1, peak_min, 0.5*(min_ciclo1 + max_ciclo1))
    peak_max = min_ciclo1 + 1*(max_ciclo1 - min_ciclo1)
    max_i = np.random.triangular(0.5*(min_ciclo1 + max_ciclo1), peak_max, max_ciclo1)
    min_max = np.array([min_i, max_i])
    noise_amplitude = 0.15*np.absolute(np.amax(min_max) - np.amin(min_max))
    matrix_features[:, i+2] = feature_up(np.amin(min_max), np.amax(min_max), N, N*0.2, noise_amplitude)

    # FEATURE 6: DECREASING
    i = 5
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    peak_min = min_ciclo1 + 0*(max_ciclo1 - min_ciclo1)
    min_i = np.random.triangular(min_ciclo1, peak_min, 0.5*(min_ciclo1 + max_ciclo1))
    peak_max = min_ciclo1 + 1*(max_ciclo1 - min_ciclo1)
    max_i = np.random.triangular(0.5*(min_ciclo1 + max_ciclo1), peak_max, max_ciclo1)
    min_max = np.array([min_i, max_i])
    noise_amplitude = 0.15*np.absolute(np.amax(min_max) - np.amin(min_max))
    matrix_features[:, i+2] = feature_down(np.amax(min_max), np.amin(min_max), N, N*0.2, noise_amplitude)

    # FEATURE 7: DUAL = RISING OR DECREASING
    i = 6
    min_ciclo1 = np.amin(MIN_MAX[i, :])
    max_ciclo1 = np.amax(MIN_MAX[i, :])
    noise_amplitude = 0.05*np.absolute(max_ciclo1 - min_ciclo1)
    matrix_features[:, i+2] = feature_dual(min_ciclo1, max_ciclo1, 0.2, N, N*0.2, noise_amplitude)

    # DATAFRAME
    df = pd.DataFrame(matrix_features)
    df[[0, 1, N_features + 2, N_features + 3]] = df[[0, 1, N_features + 2, N_features + 3]].astype('int32')
    
    dataset = dataset.append(df, ignore_index=True)

In [ ]:
# REPLACE TTF LABELS
dataset.loc[:, N_features + 3].replace({0: 'long', 1: 'short'}, inplace=True)
header = ['engine_ID', 'cycle', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 'RUL', 'TTF']
dataset.columns = header

# SAVE TO FILE
date_time_now = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
filename = 'C:/data_sim_' + date_time_now + '.csv'
#dataset.to_csv(filename, index=False, header=header)

In [8]:
# INFORMATION ABOUT THE CREATED EVENT HUBS NAMESPACE AND EVENT HUB
ADDRESS = 'xxxxx' # example 'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1'
USER = 'RootManageSharedAccessKey'
KEY = 'xxxxx' # Primary key

In [9]:
#client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
#sender = client.add_sender(partition="0")
#client.run()

In [10]:
dataset_dict = dataset.iloc[:, 0:9].astype(object).to_dict(orient='records', into=OrderedDict)

for i in range(dataset.shape[0]):
    
    datax = json.dumps(dataset_dict[i])
    print(datax)
    time.sleep(0.1)

    #sender.send(EventData(datax))

{"0": 1, "1": 1, "2": 641.8685110957615, "3": 553.8138759780124, "4": 47.158378481211464, "5": 393.0, "6": 1377.671567374598, "7": 39.11275568164094, "8": 8114.594778680788}
{"0": 1, "1": 2, "2": 642.0943528782661, "3": 553.603369379088, "4": 46.91197938039042, "5": 394.0, "6": 1387.276106293442, "7": 39.28770523590437, "8": 8111.5021708557715}
{"0": 1, "1": 3, "2": 641.8129741011461, "3": 553.5491343168123, "4": 46.92496149001714, "5": 394.0, "6": 1399.137969028342, "7": 39.290886700043764, "8": 8113.301738116386}
{"0": 1, "1": 4, "2": 642.0006110576451, "3": 554.3311550524008, "4": 46.771728306682164, "5": 392.0, "6": 1387.9334037470046, "7": 39.04901181113445, "8": 8110.481721434621}
{"0": 1, "1": 5, "2": 642.3539886499573, "3": 553.768489485453, "4": 47.119385057916425, "5": 394.0, "6": 1404.3097520024983, "7": 38.72208991162175, "8": 8112.470975821081}
{"0": 1, "1": 6, "2": 641.8557664131293, "3": 553.4459771763928, "4": 46.75763042132545, "5": 393.0, "6": 1402.6904989934978, "7":

{"0": 1, "1": 48, "2": 641.8833078878519, "3": 553.9623296515273, "4": 47.30166091017493, "5": 392.0, "6": 1385.5498754885543, "7": 39.109290063478596, "8": 8111.109112820928}
{"0": 1, "1": 49, "2": 641.4793255944059, "3": 553.4701729975949, "4": 47.377836444677676, "5": 393.0, "6": 1396.4386518027557, "7": 39.27124866821422, "8": 8110.985615045182}
{"0": 1, "1": 50, "2": 642.1671298713114, "3": 553.4715368257682, "4": 47.17357471777299, "5": 393.0, "6": 1396.3288403396784, "7": 39.029969732375314, "8": 8108.9879385051845}
{"0": 1, "1": 51, "2": 641.6856474922502, "3": 553.8498179878123, "4": 47.139100358842384, "5": 393.0, "6": 1383.4970259524177, "7": 39.19790391998476, "8": 8113.077822065458}
{"0": 1, "1": 52, "2": 641.782023568095, "3": 554.0074668526186, "4": 46.919343609649616, "5": 394.0, "6": 1401.6062198854515, "7": 39.29843158180344, "8": 8110.886979118687}
{"0": 1, "1": 53, "2": 641.5163897659834, "3": 554.7633031763219, "4": 46.93718582681013, "5": 392.0, "6": 1403.34005227

{"0": 1, "1": 95, "2": 641.5979973148873, "3": 553.3518664782641, "4": 47.34318697357656, "5": 392.0, "6": 1399.6432903108362, "7": 39.32249127428557, "8": 8112.600713268989}
{"0": 1, "1": 96, "2": 641.9887158447959, "3": 553.4375435452461, "4": 47.385894805687464, "5": 394.0, "6": 1388.8304433775509, "7": 39.30614009831585, "8": 8109.594209124818}
{"0": 1, "1": 97, "2": 642.0716917355361, "3": 553.6901397720068, "4": 47.63703792909558, "5": 394.0, "6": 1395.5989130436258, "7": 39.39546708746322, "8": 8108.2471348159925}
{"0": 1, "1": 98, "2": 641.8578805245181, "3": 553.4534109517909, "4": 47.33392263937337, "5": 393.0, "6": 1395.720862533229, "7": 39.096115123891245, "8": 8109.324338031926}
{"0": 1, "1": 99, "2": 642.2807505912525, "3": 553.2876660857656, "4": 47.415294802063045, "5": 393.0, "6": 1401.8832220403349, "7": 39.0658023828674, "8": 8108.700107169053}
{"0": 1, "1": 100, "2": 642.0791440569619, "3": 553.6823965694015, "4": 46.943745515264226, "5": 395.0, "6": 1388.200358963

{"0": 1, "1": 144, "2": 641.7386201425668, "3": 553.0691318112806, "4": 47.33860562925429, "5": 394.0, "6": 1390.0895632805352, "7": 39.15415452935175, "8": 8104.418416172493}
{"0": 1, "1": 145, "2": 641.9655288970197, "3": 552.7304192081571, "4": 47.52147843197685, "5": 394.0, "6": 1388.594074937737, "7": 38.93452080958465, "8": 8105.01306554299}
{"0": 1, "1": 146, "2": 642.3894131341845, "3": 552.1153204330552, "4": 47.91700671660541, "5": 395.0, "6": 1410.5656724083813, "7": 38.99708073790744, "8": 8103.418741703723}
{"0": 1, "1": 147, "2": 642.6278701114571, "3": 552.76161188306, "4": 47.466615312546104, "5": 394.0, "6": 1395.0729608600286, "7": 39.01499458466233, "8": 8101.798691285262}
{"0": 1, "1": 148, "2": 642.742760859841, "3": 553.0977684880486, "4": 47.08902762827749, "5": 394.0, "6": 1407.0566664957896, "7": 38.70261578514827, "8": 8108.124830671226}
{"0": 1, "1": 149, "2": 641.9097356689861, "3": 552.5622286848658, "4": 47.14198775459429, "5": 394.0, "6": 1393.85917717340

{"0": 1, "1": 191, "2": 643.7775831332036, "3": 551.2392388505465, "4": 48.489501691727504, "5": 396.0, "6": 1434.6058037800315, "7": 38.187162130750494, "8": 8088.141018097792}
{"0": 1, "1": 192, "2": 643.7498581205177, "3": 551.3962278807545, "4": 48.20548328403909, "5": 395.0, "6": 1437.8160627801337, "7": 38.29051940958656, "8": 8087.19722640057}
{"0": 1, "1": 193, "2": 643.9719668862813, "3": 551.3158135908254, "4": 48.65190819591878, "5": 397.0, "6": 1429.74439966273, "7": 38.238295869513095, "8": 8086.053403986175}
{"0": 1, "1": 194, "2": 644.1491979962508, "3": 551.1439899596487, "4": 48.35754057048152, "5": 396.0, "6": 1440.8439972806668, "7": 38.33094308098671, "8": 8085.755669297769}
{"0": 2, "1": 1, "2": 641.5283447648366, "3": 554.3876812895411, "4": 47.086615194406036, "5": 392.0, "6": 1412.0399041138883, "7": 38.79299942254172, "8": 8148.194077089866}
{"0": 2, "1": 2, "2": 641.4958324689483, "3": 554.7611895538915, "4": 46.94544230711888, "5": 391.0, "6": 1407.5926805551

...

{"0": 9, "1": 167, "2": 642.3390754312973, "3": 553.0024490515264, "4": 47.94190938943986, "5": 393.0, "6": 1425.3458603786312, "7": 38.66254450552431, "8": 8191.647948109837}
{"0": 9, "1": 168, "2": 642.1916722776691, "3": 552.9474406500585, "4": 47.755753159388604, "5": 391.0, "6": 1404.7583554032037, "7": 38.72981076100361, "8": 8193.827256395021}
{"0": 9, "1": 169, "2": 642.5979432109217, "3": 553.4216516315018, "4": 47.577526883065104, "5": 394.0, "6": 1428.7890783637893, "7": 39.05395131404954, "8": 8194.93296293342}
{"0": 9, "1": 170, "2": 642.2567043927909, "3": 552.3815486951468, "4": 47.586677394098984, "5": 394.0, "6": 1413.7948609818625, "7": 38.77904972610869, "8": 8194.98678200112}
{"0": 9, "1": 171, "2": 642.2945702945784, "3": 552.7959698661834, "4": 47.42280534290595, "5": 393.0, "6": 1415.7896546882898, "7": 38.604725176724706, "8": 8192.746335499489}
{"0": 9, "1": 172, "2": 642.4166250881012, "3": 552.7102863529493, "4": 47.420651447326364, "5": 393.0, "6": 1425.3618

{"0": 10, "1": 28, "2": 641.2971178034078, "3": 554.8082773126965, "4": 47.13307494382406, "5": 392.0, "6": 1392.232767384431, "7": 39.43517842757299, "8": 8118.578874595227}
{"0": 10, "1": 29, "2": 641.4382029625862, "3": 554.5563057557458, "4": 46.446199024469394, "5": 392.0, "6": 1383.1105868143839, "7": 39.380787984421495, "8": 8114.137552580249}
{"0": 10, "1": 30, "2": 641.224495225178, "3": 554.8839813385991, "4": 46.68308542748423, "5": 392.0, "6": 1397.8091850436062, "7": 40.01476327257418, "8": 8114.547953236512}
{"0": 10, "1": 31, "2": 641.9242190807573, "3": 554.3120793706317, "4": 46.4560703203725, "5": 393.0, "6": 1387.7481089311602, "7": 39.224100918905286, "8": 8115.87907133437}
{"0": 10, "1": 32, "2": 641.901673272268, "3": 554.7571949212705, "4": 46.620546253486815, "5": 395.0, "6": 1387.8005014644164, "7": 39.29851921167374, "8": 8117.485679703696}
{"0": 10, "1": 33, "2": 641.4545376917114, "3": 555.0353774437417, "4": 47.21432587266796, "5": 393.0, "6": 1388.59264104

{"0": 10, "1": 76, "2": 642.032196220178, "3": 554.4092070328614, "4": 47.16767154841199, "5": 393.0, "6": 1400.344010688801, "7": 39.48775555723108, "8": 8119.183131440632}
{"0": 10, "1": 77, "2": 641.1023556206715, "3": 554.6689462091271, "4": 46.93663298634499, "5": 395.0, "6": 1391.224155495516, "7": 39.30276635605526, "8": 8112.298773388036}
{"0": 10, "1": 78, "2": 641.7572399882412, "3": 553.7799679459372, "4": 46.878939959464596, "5": 394.0, "6": 1395.8242748673547, "7": 39.57643328310494, "8": 8116.424193023888}
{"0": 10, "1": 79, "2": 641.4757248453989, "3": 555.7245938662068, "4": 46.76994038830274, "5": 393.0, "6": 1395.2479499417536, "7": 39.26625534714036, "8": 8113.765335606481}
{"0": 10, "1": 80, "2": 641.6789015550221, "3": 554.1407710647181, "4": 46.956799043129465, "5": 393.0, "6": 1388.473720199501, "7": 39.35299438109569, "8": 8114.144986952382}
{"0": 10, "1": 81, "2": 641.4890230706364, "3": 554.6230648607619, "4": 46.82448561714306, "5": 395.0, "6": 1391.150554868

{"0": 10, "1": 125, "2": 641.9593179268385, "3": 554.7479049048571, "4": 47.11676520812058, "5": 394.0, "6": 1385.6354902089681, "7": 38.922693955692644, "8": 8111.553447350919}
{"0": 10, "1": 126, "2": 642.1169555297314, "3": 554.3961993148336, "4": 46.92117071535485, "5": 394.0, "6": 1397.3905175318564, "7": 39.364853814673104, "8": 8112.0814427828755}
{"0": 10, "1": 127, "2": 642.3667170404309, "3": 553.7240455691598, "4": 46.63965415748501, "5": 393.0, "6": 1392.9690474359477, "7": 39.31628392136166, "8": 8114.885234142025}
{"0": 10, "1": 128, "2": 641.7650039177147, "3": 554.9573378604607, "4": 46.83221952306508, "5": 394.0, "6": 1397.4920916810647, "7": 39.141656289424205, "8": 8110.189646724967}
{"0": 10, "1": 129, "2": 641.8045823910546, "3": 554.5619378910336, "4": 47.020251030636246, "5": 393.0, "6": 1397.684540017428, "7": 38.90661940939742, "8": 8109.20608921829}
{"0": 10, "1": 130, "2": 641.7091373509979, "3": 554.9762349062406, "4": 46.93242545465303, "5": 394.0, "6": 139

{"0": 10, "1": 172, "2": 642.5555953573463, "3": 552.4069021199755, "4": 47.109738831955404, "5": 397.0, "6": 1407.2756082152443, "7": 38.96615090477546, "8": 8103.56188679743}
{"0": 10, "1": 173, "2": 642.8203495959478, "3": 552.6430092720135, "4": 47.59660358203791, "5": 397.0, "6": 1410.2128339524418, "7": 38.91833511226258, "8": 8103.494948825048}
{"0": 10, "1": 174, "2": 641.846411531585, "3": 553.14597644503, "4": 47.98591282054463, "5": 396.0, "6": 1409.36537401034, "7": 38.750640114036116, "8": 8103.98411485569}
{"0": 10, "1": 175, "2": 643.0724894955463, "3": 553.1429557847318, "4": 47.655845976247114, "5": 396.0, "6": 1413.9209335433777, "7": 38.243168656932085, "8": 8101.188528903358}
{"0": 10, "1": 176, "2": 642.8073179754457, "3": 553.6154508414825, "4": 47.51973220474997, "5": 396.0, "6": 1406.1064971438275, "7": 38.738190752660294, "8": 8102.369762563369}
{"0": 10, "1": 177, "2": 642.4032633744947, "3": 552.833600750766, "4": 47.25662046250175, "5": 397.0, "6": 1415.3342

In [ ]:
#client.stop()